Ahora un desafio empresarial
en el noteboock Project3-week3.ipynb vas a crear un  proyecto generador de datos sinteticos
- Escribir modelos que pueden generar datasets
- Crea una interface con gradio UI para el producto.
- Usa los modelos de Hugginface que ya tengo acceso
-- https://huggingface.co/collections/meta-llama/metas-llama-32-language-models-and-evals
-- https://huggingface.co/collections/google/googles-gemma-models-family
-- https://huggingface.co/collections/meta-llama/metas-llama-31-models-and-evals
